# EOD Price History Prediction
*This notebook is intended to predict price movements based on EOD closing prices. At this time, this notebook will need to be paused, as there were some errors in data collection.*

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sqlite3
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

In [4]:
conn = sqlite3.connect('stockPrediction_05262021.db')

query = "SELECT r.stock_symbol, l.price_datetime, l.open_price, l.high_price, l.low_price, l.close_price, l.volume, l.dividends, l.stock_splits FROM eod_price_history l INNER JOIN stock r ON r.stock_id = l.stock_id;"

symbols = conn.execute('SELECT stock_symbol FROM stock')
symbols = symbols.fetchall()
symbols = [i[0] for i in symbols]
symbols = [i for i in symbols if i not in symbols]

# columns = pd.MultiIndex.from_product([[i for i in symbols], ['open_price', 'high_price', 'low_price', 'close_price']], names=['symbol', 'type'])

df = pd.read_sql(query, conn, index_col=['stock_symbol', 'price_datetime'])

# Two

# remove stocks w/ less than 1000 data points
s = df.groupby(level=0).size()
# s = s.where(s > 1000).dropna()
s = s.where(s > 1550).dropna()  # playing around w/ different values here to get a model that can actually be trained
dfs = s.to_frame()
dfs = dfs.reset_index()
df = df.reset_index()
# dfs

# Three
df = pd.merge(df, dfs, how='inner', left_on='stock_symbol', right_on='stock_symbol', left_index=True)
df = df.drop_duplicates()
# this line below added in on 5/14 22:37:
df = df.drop(columns=[0])

# Four

# df = df.drop_duplicates(subset=['price_datetime'])
df = df.set_index(['price_datetime', 'stock_symbol']).unstack(['stock_symbol'])
df

open_price                                                  \
stock_symbol         AACG    AAGFF  AAIC  AAME   AAU  ABEO    ABEPF  ABEV   
price_datetime                                                              
1972-06-01            NaN      NaN   NaN   NaN   NaN   NaN      NaN   NaN   
1972-06-02            NaN      NaN   NaN   NaN   NaN   NaN      NaN   NaN   
1972-06-05            NaN      NaN   NaN   NaN   NaN   NaN      NaN   NaN   
1972-06-06            NaN      NaN   NaN   NaN   NaN   NaN      NaN   NaN   
1972-06-07            NaN      NaN   NaN   NaN   NaN   NaN      NaN   NaN   
...                   ...      ...   ...   ...   ...   ...      ...   ...   
2021-05-20           2.83  0.39800  3.95  4.07  0.55  1.50  0.10800  3.34   
2021-05-21           2.89  0.47000  4.05  4.20  0.56  1.60  0.09300  3.35   
2021-05-24           2.92  0.49100  4.09  4.38  0.54  1.57  0.09200  3.32   
2021-05-25           2.84  0.49401  4.02  4.14  0.55  1.54  0.10470  3.36   
2021-05-26           2.84  0.47200  4.05  4.14  0.55  1.54  0.11902  3.36   

                               ... stock_splits                            \
stock_symbol    ABIO     ABMC  ...         YUII  YVR ZDPY ZIMCF ZIOP ZIVO   
price_datetime                 ...                                          
1972-06-01       NaN      NaN  ...          NaN  NaN  NaN   NaN  NaN  NaN   
1972-06-02       NaN      NaN  ...          NaN  NaN  NaN   NaN  NaN  NaN   
1972-06-05       NaN      NaN  ...          NaN  NaN  NaN   NaN  NaN  NaN   
1972-06-06       NaN      NaN  ...          NaN  NaN  NaN   NaN  NaN  NaN   
1972-06-07       NaN      NaN  ...          NaN  NaN  NaN   NaN  NaN  NaN   
...              ...      ...  ...          ...  ...  ...   ...  ...  ...   
2021-05-20      3.34  0.06000  ...            0  0.0  0.0   0.0    0    0   
2021-05-21      3.37  0.07000  ...            0  0.0  0.0   0.0    0    0   
2021-05-24      3.34  0.06000  ...            0  0.0  0.0   0.0    0    0   
2021-05-25      3.25  0.06120  ...            0  0.0  0.0   0.0    0    0   
2021-05-26      3.25  0.05965  ...          NaN  0.0  0.0   NaN    0    0   

                                     
stock_symbol   ZMWYF ZNOG ZSAN ZTNO  
price_datetime                       
1972-06-01       NaN  NaN  NaN  NaN  
1972-06-02       NaN  NaN  NaN  NaN  
1972-06-05       NaN  NaN  NaN  NaN  
1972-06-06       NaN  NaN  NaN  NaN  
1972-06-07       NaN  NaN  NaN  NaN  
...              ...  ...  ...  ...  
2021-05-20       0.0    0  0.0  0.0  
2021-05-21       0.0    0  0.0  0.0  
2021-05-24       0.0    0  0.0  0.0  
2021-05-25       0.0    0  0.0  0.0  
2021-05-26       0.0    0  0.0  0.0  

[12357 rows x 9786 columns]

In [5]:
df = df.reset_index()
df['price_datetime'] = pd.to_datetime(df['price_datetime'])
df = df.set_index('price_datetime')

In [6]:
df

open_price                                                  \
stock_symbol         AACG    AAGFF  AAIC  AAME   AAU  ABEO    ABEPF  ABEV   
price_datetime                                                              
1972-06-01            NaN      NaN   NaN   NaN   NaN   NaN      NaN   NaN   
1972-06-02            NaN      NaN   NaN   NaN   NaN   NaN      NaN   NaN   
1972-06-05            NaN      NaN   NaN   NaN   NaN   NaN      NaN   NaN   
1972-06-06            NaN      NaN   NaN   NaN   NaN   NaN      NaN   NaN   
1972-06-07            NaN      NaN   NaN   NaN   NaN   NaN      NaN   NaN   
...                   ...      ...   ...   ...   ...   ...      ...   ...   
2021-05-20           2.83  0.39800  3.95  4.07  0.55  1.50  0.10800  3.34   
2021-05-21           2.89  0.47000  4.05  4.20  0.56  1.60  0.09300  3.35   
2021-05-24           2.92  0.49100  4.09  4.38  0.54  1.57  0.09200  3.32   
2021-05-25           2.84  0.49401  4.02  4.14  0.55  1.54  0.10470  3.36   
2021-05-26           2.84  0.47200  4.05  4.14  0.55  1.54  0.11902  3.36   

                               ... stock_splits                            \
stock_symbol    ABIO     ABMC  ...         YUII  YVR ZDPY ZIMCF ZIOP ZIVO   
price_datetime                 ...                                          
1972-06-01       NaN      NaN  ...          NaN  NaN  NaN   NaN  NaN  NaN   
1972-06-02       NaN      NaN  ...          NaN  NaN  NaN   NaN  NaN  NaN   
1972-06-05       NaN      NaN  ...          NaN  NaN  NaN   NaN  NaN  NaN   
1972-06-06       NaN      NaN  ...          NaN  NaN  NaN   NaN  NaN  NaN   
1972-06-07       NaN      NaN  ...          NaN  NaN  NaN   NaN  NaN  NaN   
...              ...      ...  ...          ...  ...  ...   ...  ...  ...   
2021-05-20      3.34  0.06000  ...            0  0.0  0.0   0.0    0    0   
2021-05-21      3.37  0.07000  ...            0  0.0  0.0   0.0    0    0   
2021-05-24      3.34  0.06000  ...            0  0.0  0.0   0.0    0    0   
2021-05-25      3.25  0.06120  ...            0  0.0  0.0   0.0    0    0   
2021-05-26      3.25  0.05965  ...          NaN  0.0  0.0   NaN    0    0   

                                     
stock_symbol   ZMWYF ZNOG ZSAN ZTNO  
price_datetime                       
1972-06-01       NaN  NaN  NaN  NaN  
1972-06-02       NaN  NaN  NaN  NaN  
1972-06-05       NaN  NaN  NaN  NaN  
1972-06-06       NaN  NaN  NaN  NaN  
1972-06-07       NaN  NaN  NaN  NaN  
...              ...  ...  ...  ...  
2021-05-20       0.0    0  0.0  0.0  
2021-05-21       0.0    0  0.0  0.0  
2021-05-24       0.0    0  0.0  0.0  
2021-05-25       0.0    0  0.0  0.0  
2021-05-26       0.0    0  0.0  0.0  

[12357 rows x 9786 columns]